# Prep Data

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
folders = os.listdir('decomission_sample_data_scwrl_no_water')

## Encodings

In [3]:
res_map = {None: 0,'Ala': 1,'Arg': 2,
          'Asn': 3,'Asp': 4,'Cys': 5,
          'Glu': 6, 'Gln': 7,'Gly': 8,
          'His': 9, 'Ile': 10,'Leu': 11,
          'Lys': 12,'Met': 13,'Phe': 14,
          'Pro': 15,'Ser': 16,'Thr': 17,
          'Trp': 18,'Tyr': 19,'Val': 20 }

ele_map = {'O': 1, 'N': 2, 'C': 3, 'S': 4, 'P':5, 'Cl': 6, 'F':7}

origin_map = {'het': 0, 'prot': 1, 'lig': 2}

In [4]:
import json
import pandas as pd

root = 'decomission_sample_data_scwrl_no_water'

data = {'element_type':[], 'residue_type': [], 'hydrophobic_interactions': [],
        'hydrogen_bonds': [], 'water_bridges':[], 'pi_stacks': [],
        'pi_cation_interactions': [], 'halogen_bonds':[], 'salt_bridges':[],
        'metal_complexes':[], 'origin_type':[], 'complex_code':[]
       }


for folder in folders:
    
    with open(f'{root}/{folder}/{folder}_datamap.json') as f:
        feature_map = json.load(f)
    
    if len(feature_map) == 0:
        print(folder)
        continue
    #print(folder)
    #print(feature_map)
    for atom in feature_map:
            
        features = feature_map[atom]
        if features['element_type'] is None:
            continue
        
        element_encoding = ele_map[features['element_type']]
        
        if features['residue_type'] is None:
            residue_encoding = res_map[features['residue_type']]
                
        if isinstance(features['residue_type'], str):
            residue_encoding = res_map[features['residue_type'].capitalize().strip()]
                
        origin_encoding  = origin_map[features['origin_type'].split('_')[0]]
            
        data['element_type'].append(element_encoding)
        data['residue_type'].append(residue_encoding)
        data['origin_type'].append(origin_encoding)
        
        data['hydrophobic_interactions'].append(features['hydrophobic_interactions'])
        data['hydrogen_bonds'].append(features['hydrogen_bonds'])
        data['water_bridges'].append(features['water_bridges'])
        data['pi_stacks'].append(features['pi_stacks'])
        data['pi_cation_interactions'].append(features['pi_cation_interactions'])
        data['halogen_bonds'].append(features['halogen_bonds'])
        data['salt_bridges'].append(features['salt_bridges'])
        data['metal_complexes'].append(features['metal_complexes'])
            
        data['complex_code'].append(folder.replace('_complex', ''))
            
full_df = pd.DataFrame.from_dict(data)
full_df.to_csv('batch.csv', index=False)

CYP3A4-T309A_848780_complex


In [5]:
import torch
class GraphNN_Attention(torch.nn.Module):
    
    def __init__(self, drop_out, num_classes, embedding_dim=3, non_linearity=torch.nn.ReLU()):
        
        super(GraphNN_Attention, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        torch.manual_seed(3)
        self.W_1 = torch.nn.Linear(in_features=10, out_features=self.embedding_dim)
        torch.manual_seed(3)
        self.Y_1 = torch.nn.Linear(in_features=10, out_features=self.embedding_dim)
        
        self.non_linearity = non_linearity
        
        torch.manual_seed(3)
        self.W_attent = torch.nn.Linear(in_features=10, out_features=1)

        self.complex_embedding_map = {}
        self.atom_embedding_map = {}
        
        #classifier
        torch.manual_seed(3)
        self.layer_1 = torch.nn.Linear(embedding_dim, 5)
        torch.manual_seed(3)
        self.layer_2 = torch.nn.Linear(5, 5)
        torch.manual_seed(3)
        self.layer_out = torch.nn.Linear(5, num_classes)
        
        self.elu = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(p=drop_out)
        
        self.batchnorm1 = torch.nn.BatchNorm1d(5)
        self.batchnorm2 = torch.nn.BatchNorm1d(5)
        self.softmax = torch.nn.Softmax(dim=1)
        
    def attention_coeff(self, euclid_dist_v_i, i_neighbour_vec):

        #i_neighbour_vec = i_neighbour_vec.view(1, i_neighbour_vec.shape[0])
        
        return (1/euclid_dist_v_i) * self.W_attent(i_neighbour_vec)
    
    def attention_mechanism(self, ori_adj_mat, feature_mat):
        
        attention_coeff_mat = torch.zeros(ori_adj_mat.shape).cuda()
        #processed_set = set()
        softmax = torch.nn.Softmax(dim=1)
        
        for v in range(ori_adj_mat.shape[0]):

            for i in range(ori_adj_mat.shape[1]):
                
                if v == i:
                    #processed_set.add((v, i))
                    continue
                    
                attention_coeff_v_i = self.attention_coeff(ori_adj_mat[v,i], feature_mat[i,:])
                
                attention_coeff_mat[v, i] = attention_coeff_v_i
                #processed_set.add((v, i))
        
        attention_mat = softmax(attention_coeff_mat)
        
        return attention_mat

    def forward(self, adj_mat, feature_mat, complex_code):
        """
            adj_mat (torch tensor): adjacency matrix
            feature_mat (torch tensor): feature_matrix for all atoms
            complex_code (str): cyp_ligand code
            return a tensor of predicted probabilities
            
        """
        #graph nn embedding procedure
        num_neighbours = adj_mat.shape[0] - 1 #since adj mat is square and symmetric

        attent_mat = self.attention_mechanism(adj_mat, feature_mat)

        avr_neighbour_feature_linear_comb = (1/num_neighbours) * (torch.mm(attent_mat, feature_mat))

        #h_v_matrix = self.non_linearity(torch.mm(avr_neighbour_feature_linear_comb, self.W_1) + torch.mm(feature_mat,self.Y_1))
        h_v_matrix = self.non_linearity(self.W_1(avr_neighbour_feature_linear_comb) + self.Y_1(feature_mat))
        
        ## persistence
        #each row of h_v_matrix == atom embedding, entire matrix == for all atoms
        self.atom_embedding_map[complex_code] = h_v_matrix 

        #self.complex_embedding_map[complex_code] = torch.mean(h_v_matrix, dim=0)
        #complex_embedding = torch.mean(h_v_matrix, dim=0)
        
        #return prob_tensor
        prob = self.elu(self.layer_1(h_v_matrix))
        
        prob = self.batchnorm1(prob)
        
        prob = self.elu(self.layer_2(prob))
        
        prob = self.batchnorm2(prob)
        
        prob = self.dropout(prob)
        
        #return complex_embedding.view(1, complex_embedding.shape[0]), self.layer_out(prob)
        
        return self.softmax(self.layer_out(prob))

In [6]:
##credits attributable to: https://debuggercafe.com/using-learning-rate-scheduler-and-early-stopping-with-pytorch/
class EarlyStopping:
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        
        if self.best_loss == None:
            self.best_loss = val_loss
            
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np

train_code, val_code = train_test_split(full_df.iloc[:, -1].unique(), test_size=0.2, random_state=42)

np.save('train_code.np', train_code)
np.save('val_code.np', val_code)

In [8]:
print(train_code)
print(val_code)

['CYP3A4-A305S_3016' 'CYP3A4-I301A_566440' 'CYP3A4-I369V_4993'
 'CYP3A4-I120F_5994' 'CYP3A4.12_6013' 'CYP3A4-T363M_6013'
 'CYP3A4-S119W_3035604' 'CYP3A4-A305F_3035604' 'CYP3A4-I369V_566440'
 'CYP3A4-F304W_11790' 'CYP3A4-A305G_4993' 'CYP3A4-T309V_6013'
 'CYP3A4-A370V_4993' 'CYP3A4-S119A_3035604' 'CYP3A4-S119A_6013'
 'CYP3A4-T103A_5994' 'CYP3A4-C98W_4192' 'CYP3A4-S119A_3016'
 'CYP3A4.11_5994' 'CYP3A4.12_5994' 'CYP3A4-S119T_6013' 'CYP3A4-L373F_5994'
 'CYP3A4-I369V_6013' 'CYP3A4-M371I_14403' 'CYP3A4.12_848780'
 'CYP3A4-I301F_3016' 'CYP3A4.3_848780' 'CYP3A4-I301A_5994'
 'CYP3A4-I369F_3035604' 'CYP3A4-S119C_4192' 'CYP3A4-S119V_6013'
 'CYP3A4-T309C_4192' 'CYP3A4-I301F_5994' 'CYP3A4-I369V_14403'
 'CYP3A4-A305V_4993' 'CYP3A4-R212A_6013' 'CYP3A4-F304A_566440'
 'CYP3A4-I301W_5994' 'CYP3A4-F304W_4192' 'CYP3A4.3_2730'
 'CYP3A4-T309V_3035604' 'CYP3A4-A305G_6013' 'CYP3A4-I301A_3016'
 'CYP3A4-R212A_5994' 'CYP3A4-N104D_5994' 'CYP3A4-A370F_3035604'
 'CYP3A4-S119A_55245' 'CYP3A4-A305V_6013' 'CYP3A4.3_601

In [11]:
np.load('val_code.np.npy', allow_pickle=True)

array(['CYP3A4-S119A_5994', 'CYP3A4-L373A_4993', 'CYP3A4-A370F_55245',
       'CYP3A4-S119F_3016', 'CYP3A4-F304A_4192', 'CYP3A4-S312C_4192',
       'CYP3A4-A305V_566440', 'CYP3A4-A305G_566440', 'CYP3A4-C98A_4192',
       'CYP3A4-I120W_4192', 'CYP3A4-T309F_4192', 'CYP3A4-F304W_6013',
       'CYP3A4-I120L_14403', 'CYP3A4.3_4485', 'CYP3A4-I120W_5994',
       'CYP3A4.11_6013', 'CYP3A4-V376T_6013', 'CYP3A4-T309A_3035604',
       'CYP3A4-V376T_14403'], dtype=object)

In [12]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau 

tb = SummaryWriter('gnn_runs/expt_17_refit_with_embeddings')

gnn = GraphNN_Attention(drop_out=0.01, num_classes=7, embedding_dim=10).cuda()
loss = torch.nn.CrossEntropyLoss()
#early_stopper = EarlyStopping(min_delta=0.01)

mean_losses = []
mean_val_losses = []

epochs = 100
learning_rate = 0.05

optimiser = torch.optim.Adam(gnn.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimiser, mode='min', verbose=True)
root_folder = 'decomission_sample_data_scwrl_no_water'

train_complex_codes = list(train_code)
val_complex_codes = list(val_code)
bad_train_data = set()
bad_val_data = set()

is_visited = set()

for i in range(epochs):
    
    losses = []
    
    ## training loop
    for train_code in train_complex_codes:
        if train_code == 'C':
            continue
            
        folder = f'{train_code}_complex'
                    
        adj_mat_path = f'{root_folder}/{folder}/{folder}_adjmat.npy'
        adj_mat = np.load(adj_mat_path)
        adj_mat_torch = torch.from_numpy(adj_mat).float().cuda()
        
        x_train = full_df[full_df['complex_code'] == train_code]
        y_train = x_train.iloc[:,0].to_numpy()
        x_train = x_train.iloc[:, 1:-1].to_numpy()
                
        if train_code not in is_visited:
            feature_mat_torch = torch.from_numpy(x_train).float().cuda()
            is_visited.add(train_code)
            
        elif train_code in is_visited:
            prev_embeddings = gnn.atom_embedding_map[folder].detach().cpu().numpy()
            feature_mat_torch = torch.from_numpy(prev_embeddings).float().cuda()

        prob = gnn(adj_mat=adj_mat_torch, feature_mat=feature_mat_torch, complex_code=folder)
        output = loss(prob, torch.tensor(y_train).cuda())
        output.backward()
        losses.append(output.item())
        
    optimiser.step()
    optimiser.zero_grad()
    
    mean_loss = sum(losses)/len(losses)
    mean_losses.append(mean_loss)
    
    ## validation loop
    curr_val_losses = []
    
    for val_code in val_complex_codes:
        val_folder = f'{val_code}_complex'
        adj_mat_path = f'{root_folder}/{val_folder}/{val_folder}_adjmat.npy'
        adj_mat = np.load(adj_mat_path)
        adj_mat_torch = torch.from_numpy(adj_mat).float().cuda()
    
        x_val = full_df[full_df['complex_code'] == val_code]
        y_val = x_val.iloc[:,0].to_numpy()
        x_val = x_val.iloc[:, 1:-1].to_numpy()
        

        feature_mat_torch = torch.from_numpy(x_val).float().cuda()
        
        if adj_mat_torch.shape[0] != feature_mat_torch.shape[0]:
            bad_val_data.add(val_code)
            continue
        
        
        prob = gnn(adj_mat=adj_mat_torch, feature_mat=feature_mat_torch, complex_code=val_folder)
        curr_val_losses.append(loss(prob, torch.tensor(y_val).cuda()).item())
    
    mean_val_loss = sum(curr_val_losses)/len(curr_val_losses)
    mean_val_losses.append(mean_val_loss)
    
    scheduler.step(mean_val_loss)
    
    tb.add_scalar("Mean CrossEntropyLoss/train", mean_loss, i)
    tb.add_scalar("Mean CrossEntropyLoss/val", mean_val_loss, i)
    tb.add_histogram('weights/gnn w_1 weight', gnn.W_1.weight, i)
    tb.add_histogram('weights/gnn y_1 linear layer weight', gnn.Y_1.weight, i)
    tb.add_histogram('weights/gnn w_attent weight', gnn.W_attent.weight, i)
    tb.add_histogram('weights/gnn layer_1', gnn.layer_1.weight, i)
    tb.add_histogram('weights/gnn layer_2', gnn.layer_2.weight, i)
    tb.add_histogram('weights/gnn layer_out', gnn.layer_out.weight, i)
    
    print(f"epoch:{i+1} train_loss:{mean_loss} val_loss:{mean_val_loss} learning rate:{optimiser.param_groups[0]['lr']}")

epoch:1 train_loss:1.963720489854682 val_loss:1.9010195997026231 learning rate:0.05
epoch:2 train_loss:1.9109153290317482 val_loss:1.8547956612375047 learning rate:0.05
epoch:3 train_loss:1.8463108131330308 val_loss:1.809531980090671 learning rate:0.05
epoch:4 train_loss:1.7889701781207568 val_loss:1.7649690045250788 learning rate:0.05
epoch:5 train_loss:1.7557600671297884 val_loss:1.7344865997632344 learning rate:0.05
epoch:6 train_loss:1.7263740529752758 val_loss:1.7021534244219463 learning rate:0.05
epoch:7 train_loss:1.7037000786768246 val_loss:1.6741768254174128 learning rate:0.05
epoch:8 train_loss:1.6965517524170548 val_loss:1.6591516733169556 learning rate:0.05
epoch:9 train_loss:1.676902713840955 val_loss:1.62867585155699 learning rate:0.05
epoch:10 train_loss:1.6587012075398067 val_loss:1.6266485783788893 learning rate:0.05
epoch:11 train_loss:1.653182134236375 val_loss:1.604175607363383 learning rate:0.05
epoch:12 train_loss:1.6447551005507168 val_loss:1.6017682022518582 lea

epoch:83 train_loss:1.5542228483173945 val_loss:1.6866331828965082 learning rate:5.000000000000002e-07
epoch:84 train_loss:1.554226470320192 val_loss:1.6924183170000713 learning rate:5.000000000000002e-07
Epoch    85: reducing learning rate of group 0 to 5.0000e-08.
epoch:85 train_loss:1.5542859916817653 val_loss:1.6900654236475627 learning rate:5.0000000000000024e-08
epoch:86 train_loss:1.5542366325038752 val_loss:1.6856175131267972 learning rate:5.0000000000000024e-08
epoch:87 train_loss:1.554229538734645 val_loss:1.6861865056885614 learning rate:5.0000000000000024e-08
epoch:88 train_loss:1.5542275693318615 val_loss:1.684918271170722 learning rate:5.0000000000000024e-08
epoch:89 train_loss:1.554240639895609 val_loss:1.6854544281959534 learning rate:5.0000000000000024e-08
epoch:90 train_loss:1.554227593826921 val_loss:1.6855400535795424 learning rate:5.0000000000000024e-08
epoch:91 train_loss:1.5542715706237376 val_loss:1.6874333421389263 learning rate:5.0000000000000024e-08
epoch:92 

In [10]:
import json

save_dict = {}

for embedding in gnn.atom_embedding_map:
    save_dict[embedding] = gnn.atom_embedding_map[embedding].cpu().detach().numpy().tolist()
    
json_info = json.dumps(save_dict)
print(save_dict)
with open('atom_embeddings_expt_16_refit_with_embeddings.json', 'w+') as file:
    file.write(json_info)

{'CYP3A4-A305S_3016_complex': [[0.0, 2.9741358757019043, 0.0, 0.0, 3.17000150680542, 0.0, 0.8153607845306396, 0.0, 0.0, 0.0], [0.0, 2.9741358757019043, 0.0, 0.0, 3.17000150680542, 0.0, 0.8153607845306396, 0.0, 0.0, 0.0]], 'CYP3A4-I301A_566440_complex': [[0.0, 225060765827072.0, 0.0, 0.0, 0.0, 0.0, 342084313677824.0, 91243585470464.0, 0.0, 0.0], [0.0, 223751388004352.0, 0.0, 0.0, 0.0, 0.0, 340094133207040.0, 90712737579008.0, 0.0, 0.0], [0.0, 224931514155008.0, 0.0, 0.0, 0.0, 0.0, 341887852478464.0, 91191181836288.0, 0.0, 0.0], [0.0, 224625816502272.0, 0.0, 0.0, 0.0, 0.0, 341423257812992.0, 91067248541696.0, 0.0, 0.0], [0.0, 224936312438784.0, 0.0, 0.0, 0.0, 0.0, 341895167344640.0, 91193119604736.0, 0.0, 0.0], [0.0, 224624122003456.0, 0.0, 0.0, 0.0, 0.0, 341420640567296.0, 91066552287232.0, 0.0, 0.0], [0.0, 225024275382272.0, 0.0, 0.0, 0.0, 0.0, 342028881756160.0, 91228796354560.0, 0.0, 0.0], [0.0, 223918740733952.0, 0.0, 0.0, 0.0, 0.0, 340348542910464.0, 90780593029120.0, 0.0, 0.0], [0

In [15]:
import torch
class RegressionNN(torch.nn.Module):
    def __init__(self, dim=(1,22)):
        """
            dim (int) should be the shape of contact_complex_embedding
        """
        super(RegressionNN, self).__init__()
        #self.first_out = random.randint(1, 10)
        self.first_out = 10
        self.dim = dim
        
        torch.manual_seed(3)
        self.first_layer = torch.nn.Linear(in_features=self.dim[-1], out_features=self.first_out)
        
        self.first_non_linear = torch.nn.ELU()
        
        torch.manual_seed(3)
        self.second_layer = torch.nn.Linear(in_features=self.first_out, out_features=50)
        
        self.second_non_linear = torch.nn.ELU()
        
        torch.manual_seed(3)
        self.third_layer = torch.nn.Linear(in_features=50, out_features=60)
        
        self.third_non_linear = torch.nn.Sigmoid()
        
        torch.manual_seed(3)
        self.out = torch.nn.Linear(in_features=60, out_features=1)
        
    def forward(self, contact_complex_embedding):
        first_layer_out = self.first_layer(contact_complex_embedding)
        first_non_linear_out = self.first_non_linear(first_layer_out)
        
        second_layer_out = self.second_layer(first_non_linear_out)
        second_non_linear_out = self.second_non_linear(second_layer_out)
        
        third_layer_out = self.third_layer(second_non_linear_out)
        third_non_linear_out = self.third_non_linear(third_layer_out)
        
        result = self.out(third_non_linear_out)
        return result

In [33]:
import pandas as pd

db = 'test_db/single_near_mutant_CYP3A4_db.csv'
df = pd.read_csv(db)

full_df = df.loc[:,['mt_code', 'act', 'cid']]
full_df

,mt_code,act,cid
0,CYP3A4.3,202.000,4485
1,CYP3A4-S119A,29.000,6013
2,CYP3A4-S119A,4.104,6013
3,CYP3A4-S119A,0.442,6013
4,CYP3A4-S119L,2.431,6013
...,...,...,...
224,CYP3A4.11,2.900,848780
225,CYP3A4.12,4.500,848780
226,CYP3A4-F304W,2.200,11790
227,CYP3A4.3,0.120,2730


In [22]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau

tb = SummaryWriter('regression_runs/expt_16_refit_with_embeddings')

regression_epochs = 10000

regression_learning_rate = 0.05

regression_loss = torch.nn.MSELoss()

regressor = RegressionNN(dim=(1,10)).cuda()

train_complex_codes = list(np.load('train_code.np.npy', allow_pickle=True))
val_complex_codes = list(np.load('val_code.np.npy', allow_pickle=True))

optimiser = torch.optim.Adam(regressor.parameters(), lr=regression_learning_rate)
scheduler = ReduceLROnPlateau(optimiser, mode='min', verbose=True)

mean_train_losses = []
mean_val_losses = []

for epoch in range(regression_epochs):
    train_losses = []
    val_losses = []
    
    for train_code in train_complex_codes:
        train_embedding = np.array(graph_inputs[f'{train_code}_complex'])
        atom_embedding_matrix = torch.from_numpy(train_embedding).cuda().float()
        complex_embedding = torch.mean(atom_embedding_matrix, dim=0)
        y_pred = regressor(complex_embedding)
        
        mt_code, cid = train_code.split('_')
        filtered_df = full_df[full_df.mt_code.isin([mt_code]) & full_df.cid.isin([cid])]
        
        y_true = torch.tensor(filtered_df.iloc[0, 1]).view(1,1).cuda().float()
        
        train_loss = regression_loss(y_pred, y_true)
        
        train_losses.append(train_loss.item())
        train_loss.backward()
        
    for val_code in val_complex_codes:
        val_embedding = np.array(graph_inputs[f'{val_code}_complex'])
        atom_embedding_matrix = torch.from_numpy(val_embedding).cuda().float()
        complex_embedding = torch.mean(atom_embedding_matrix, dim=0)
        y_pred = regressor(complex_embedding)
        
        mt_code, cid = val_code.split('_')
        filtered_df = full_df[full_df.mt_code.isin([mt_code]) & full_df.cid.isin([cid])]
        y_true = torch.tensor(filtered_df.iloc[0, 1]).view(1,1).cuda().float()
        
        val_loss = regression_loss(y_pred, y_true)
        val_losses.append(val_loss.item())
    
        
        
    optimiser.step()
    optimiser.zero_grad()
        
    mean_train_loss = sum(train_losses)/len(train_losses)
    mean_train_losses.append(mean_train_loss)
    
    mean_val_loss = sum(val_losses)/len(val_losses)
    mean_val_losses.append(mean_val_loss)
    scheduler.step(mean_val_loss)
    
    tb.add_scalar("Mean MSE/train", mean_train_loss, epoch)
    
    tb.add_histogram('weights/regression 1st layer weight', regressor.first_layer.weight, epoch)
    tb.add_histogram('weights/regression 2nd layer weight', regressor.second_layer.weight, epoch)
    tb.add_histogram('weights/regression 3rd layer weight', regressor.third_layer.weight, epoch)
    tb.add_histogram('weights/regression out weight', regressor.out.weight, epoch)
    
    print(f'epoch:{epoch+1} Average Train MSE:{mean_train_loss} Average Val MSE:{mean_val_loss}')

KeyError: 'CYP3A4.3_4485_complex'

In [9]:
import torch
from torch import nn

torch.manual_seed(3)
linear = nn.Linear(5, 2)

torch.manual_seed(3)
linear2 = nn.Linear(5, 2)

print(linear.weight)
print(linear2.weight)

Parameter containing:
tensor([[-0.4434, -0.3528, -0.1915, -0.4231, -0.0254],
        [-0.3934,  0.2432,  0.2180,  0.0845,  0.3469]], requires_grad=True)
Parameter containing:
tensor([[-0.4434, -0.3528, -0.1915, -0.4231, -0.0254],
        [-0.3934,  0.2432,  0.2180,  0.0845,  0.3469]], requires_grad=True)


In [ ]:
Parameter containing:
tensor([[-0.4434, -0.3528, -0.1915, -0.4231, -0.0254],
        [-0.3934,  0.2432,  0.2180,  0.0845,  0.3469]], requires_grad=True)
Parameter containing:
tensor([[-0.4434, -0.3528, -0.1915, -0.4231, -0.0254],
        [-0.3934,  0.2432,  0.2180,  0.0845,  0.3469]], requires_grad=True)
